In [1]:
import glob
import h5py
import json
from tgv_teno5.tgv_postprocessing import get_jaxfluids_results, get_file_list
target_folder_name = f"tgv_teno5/results/tgv"
file = f"{target_folder_name}/domain/data_*.h5"

In [2]:
def calculate_cell_volume(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    x_range = data['domain']['x']['range']
    y_range = data['domain']['y']['range']
    z_range = data['domain']['z']['range']

    cells_x = data['domain']['x']['cells']
    cells_y = data['domain']['y']['cells']
    cells_z = data['domain']['z']['cells']

    cell_length_x = (x_range[1] - x_range[0]) / cells_x
    cell_length_y = (y_range[1] - y_range[0]) / cells_y
    cell_length_z = (z_range[1] - z_range[0]) / cells_z

    cell_volume = cell_length_x * cell_length_y * cell_length_z

    return cell_volume


In [3]:
def calculate_kinetic_energy(file, cell_volume):
    file = get_file_list(file)
    velocity = get_jaxfluids_results(file,  ["velocity"])["velocity"]
    velocity_x = velocity['velocity_x']
    velocity_y = velocity['velocity_y']
    velocity_z = velocity['velocity_z']
    print(velocity_z.shape)
    density = get_jaxfluids_results(file,  ["density"])["density"]
    print(density.shape)
    kinetic_energy = 0.5 * density * cell_volume * (velocity_x**2 + velocity_y**2 + velocity_z**2) # for 3D tgv
    print(kinetic_energy.sum())
    return kinetic_energy.sum()

In [5]:
cell_volume = calculate_cell_volume('tgv_teno5/results/tgv/tgv.json')
print(cell_volume)
kinetic_energy = calculate_kinetic_energy(file, cell_volume)

0.0009462364709564153
Load tgv_teno5/results/tgv/domain/data_10.0000948559.h5
(64, 64, 64)
Load tgv_teno5/results/tgv/domain/data_10.0000948559.h5
(64, 64, 64)
18.289785674371107
